# Import thư viện

In [11]:
import sqlite3
import math
import pandas as pd
import scipy.stats as stats
import numpy as np
import itertools

# Câu 1 : Tính hệ số tương quan giữa hai biến A và B.

In [12]:
# 1. Kết nối SQLite in-memory
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 2. Tạo bảng với 2 cột A và B
cursor.execute("""
    CREATE TABLE data (
        A REAL,
        B REAL
    );
""")

# 3. Chèn dữ liệu mẫu
sample_data = [
    (1, 2),
    (2, 4),
    (3, 5),
    (4, 4),
    (5, 5),
    (6, 7)
]

cursor.executemany("INSERT INTO data (A, B) VALUES (?, ?);", sample_data)

# 4. Truy vấn dữ liệu cần thiết cho công thức Pearson
cursor.execute("""
    SELECT 
        COUNT(*) AS n,
        SUM(A) AS sum_a,
        SUM(B) AS sum_b,
        SUM(A * B) AS sum_ab,
        SUM(A * A) AS sum_a2,
        SUM(B * B) AS sum_b2
    FROM data;
""")

n, sum_a, sum_b, sum_ab, sum_a2, sum_b2 = cursor.fetchone()

# 5. Tính hệ số tương quan Pearson
numerator = n * sum_ab - sum_a * sum_b
denominator = math.sqrt(n * sum_a2 - sum_a**2) * math.sqrt(n * sum_b2 - sum_b**2)

r_ab = numerator / denominator if denominator != 0 else None

print("Hệ số tương quan Pearson giữa A và B là:", r_ab)

# 6. Đóng kết nối
conn.close()

Hệ số tương quan Pearson giữa A và B là: 0.8783100656536799


# Câu 2 : Thực hiện kiểm định Chi-Square (χ²).

In [13]:
# Bước 1: Dữ liệu gốc
data = {
    'Day': ['Day 1', 'Day 2', 'Day 3', 'Day 4'],
    'A': [8, 7.5, 6, 7],
    'B': [9, 8.5, 7, 6],
    'C': [7, 7, 8, 5]
}

df = pd.DataFrame(data)

# Bước 2: Chuyển sang dạng quan hệ (long format)
df_long = df.melt(id_vars='Day', var_name='CarModel', value_name='Score')

# Bước 3: Phân loại điểm số (ví dụ Low <=6, Medium <=8, High >8)
def score_category(score):
    if score <= 6:
        return 'Low'
    elif score <= 8:
        return 'Medium'
    else:
        return 'High'

df_long['ScoreLevel'] = df_long['Score'].apply(score_category)

print("Dữ liệu sau khi phân loại:")
print(df_long.to_string(index=False))

# Bước 4: Tạo bảng chéo và kiểm định chi-square
contingency_table = pd.crosstab(df_long['CarModel'], df_long['ScoreLevel'])

chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

print("Bảng chéo (contingency table):\n", contingency_table)
print("\nChi-square = {:.4f}, p-value = {:.4f}, dof = {}".format(chi2, p, dof))

if p < 0.05:
    print("=> Có sự khác biệt đáng kể giữa các mẫu.")
else:
    print("=> Không có sự khác biệt đáng kể giữa các mẫu.")



Dữ liệu sau khi phân loại:
  Day CarModel  Score ScoreLevel
Day 1        A    8.0     Medium
Day 2        A    7.5     Medium
Day 3        A    6.0        Low
Day 4        A    7.0     Medium
Day 1        B    9.0       High
Day 2        B    8.5       High
Day 3        B    7.0     Medium
Day 4        B    6.0        Low
Day 1        C    7.0     Medium
Day 2        C    7.0     Medium
Day 3        C    8.0     Medium
Day 4        C    5.0        Low
Bảng chéo (contingency table):
 ScoreLevel  High  Low  Medium
CarModel                     
A              0    1       3
B              2    1       1
C              0    1       3

Chi-square = 5.1429, p-value = 0.2730, dof = 4
=> Không có sự khác biệt đáng kể giữa các mẫu.


# Câu 3 : Chuyển đổi các giá trị này thành định dạng thời gian.

In [14]:
# Ví dụ dữ liệu
df = pd.DataFrame({
    'departure_time': [830, 1445, 5, 945, 1230]
})

# Hàm chuyển đổi thời gian số sang định dạng HH:MM
def convert_to_time(x):
    x = int(x)
    hour = x // 100
    minute = x % 100
    return f"{hour:02d}:{minute:02d}"

# Áp dụng chuyển đổi
df['formatted_time'] = df['departure_time'].apply(convert_to_time)

print(df)


   departure_time formatted_time
0             830          08:30
1            1445          14:45
2               5          00:05
3             945          09:45
4            1230          12:30


# Câu 4 : Phát hiện ngoại lệ bằng phương pháp MAD (Median Absolute Deviation).

In [15]:
# Ví dụ dữ liệu
df = pd.DataFrame({
    'value': [10, 12, 11, 13, 12, 100, 11, 12, 13, 14, 9, 11]  # 100 là ngoại lệ rõ ràng
})

# Bước 1: Tính median
median = df['value'].median()

# Bước 2: Tính độ lệch tuyệt đối
df['abs_dev'] = (df['value'] - median).abs()

# Bước 3: Tính MAD
mad = df['abs_dev'].median()

# Bước 4: Tính MAD Score
df['mad_score'] = df['abs_dev'] / mad

# Bước 5: Đánh dấu ngoại lệ với ngưỡng x = 1.5
threshold = 1.5
df['is_outlier'] = df['mad_score'] > threshold

# In kết quả
print(df)


    value  abs_dev  mad_score  is_outlier
0      10      2.0        2.0        True
1      12      0.0        0.0       False
2      11      1.0        1.0       False
3      13      1.0        1.0       False
4      12      0.0        0.0       False
5     100     88.0       88.0        True
6      11      1.0        1.0       False
7      12      0.0        0.0       False
8      13      1.0        1.0       False
9      14      2.0        2.0        True
10      9      3.0        3.0        True
11     11      1.0        1.0       False


# Câu 5 : Xác định xem hai người trong bảng Patient(last_name, weight, height) có phải là cùng một người hay không.

In [16]:
# Giả sử dữ liệu bảng Patient
df = pd.DataFrame({
    'patient_id': [1, 2, 3, 4],
    'last_name': ['Nguyen', 'Nguyen', 'Tran', 'Nguyen'],
    'weight': [60, 60.5, 55, 61],
    'height': [160, 160, 165, 160]
})

# Ngưỡng cho phép sai số cân nặng (ví dụ 1.0 kg)
weight_threshold = 1.0

# So sánh từng cặp bệnh nhân
possible_matches = []
for (i1, row1), (i2, row2) in itertools.combinations(df.iterrows(), 2):
    same_last_name = row1['last_name'].lower() == row2['last_name'].lower()
    close_weight = abs(row1['weight'] - row2['weight']) <= weight_threshold

    if same_last_name and close_weight:
        possible_matches.append({
            'patient_1': row1['patient_id'],
            'patient_2': row2['patient_id'],
            'last_name': row1['last_name'],
            'weight_diff': abs(row1['weight'] - row2['weight'])
        })

# In ra các cặp nghi ngờ là cùng một người
matches_df = pd.DataFrame(possible_matches)
print("Các cặp bệnh nhân nghi là cùng một người:")
print(matches_df)


Các cặp bệnh nhân nghi là cùng một người:
   patient_1  patient_2 last_name  weight_diff
0          1          2    Nguyen          0.5
1          1          4    Nguyen          1.0
2          2          4    Nguyen          0.5
